# Homework9 by group3

Authors:Kaiyang LIN,Tianhao MOU,Lin LIU,Yunxia SHI,Guangyi NING

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Exercise 1

In [3]:
er = pd.read_csv('https://github.com/QuantEcon/ShenzhenWinterCamp/raw/master/Jiao%20Shi/ExR.csv')
erf = pd.read_csv('https://github.com/QuantEcon/ShenzhenWinterCamp/raw/master/Jiao%20Shi/ER_predict.csv')


In [4]:
# compute the change in the exchange rate: take log and first difference to compute depreciation (dep)
erf['lner'] = np.log(erf[['CNYUSD']])
erf['dep'] = 100*erf['lner'].diff(periods = 1)

In [5]:
# Create a new column to reindex depreciation - why?
erf['dep_1'] = 0
size_sample = 149

for i in range(size_sample):
    erf.loc[i,'dep_1'] = erf.loc[i+1,'dep']
    
erf.loc[size_sample,'dep_1'] = erf.loc[0,'dep']

In [6]:
erf['int_diff'] = erf['CHN_IBR'] - erf['US_FFR']

In [7]:
erf['const'] = 1 # create column of constant

reg_int = sm.OLS(endog=erf['dep_1'], exog=erf[['const', 'int_diff']], missing='drop')
results = reg_int.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  dep_1   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     4.998
Date:                Fri, 02 Feb 2018   Prob (F-statistic):             0.0269
Time:                        15:38:23   Log-Likelihood:                -148.45
No. Observations:                 149   AIC:                             300.9
Df Residuals:                     147   BIC:                             306.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1985      0.059     -3.355      0.0

In [35]:
# Now we do it recursively
size_sample = 149
size_subsample = 60

# How many regressions can we run? How many forecast errors can we compute?
fcst_error_int = np.zeros(size_sample - size_subsample)
fcst_int = np.zeros(size_sample - size_subsample)
dep_1_int = np.zeros(size_sample - size_subsample)
for i in range (size_sample - size_subsample):
    reg_intsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'int_diff']], missing='drop')
    results_sub = reg_intsub.fit()
    fcst_int[i] = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']])
    dep_1_int[i]=erf.loc[size_subsample+i, 'dep_1']
    error = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_error_int[i] = error
    
#Use the percentage of correct directional forecast as the criterion
fcst_int_direction=np.where(fcst_int>=0,1,0)
dep_1_int_direction=np.where(dep_1_int>=0,1,0)
percentage_of_correct_directional_forecast_int=np.mean(fcst_int_direction==dep_1_int_direction)
print(percentage_of_correct_directional_forecast_int)

0.61797752809


In [34]:
# Taylor-Rule model

# inflation differential
erf['pi_diff'] = erf['CHN CPI inflation'] - erf['USA CPI inflation']

fcst_tr = np.zeros(size_sample - size_subsample)
fcst_error_tr = np.zeros(size_sample - size_subsample)
dep_1_tr = np.zeros(size_sample - size_subsample)
for i in range (size_sample - size_subsample):
    # Create a column to store the output gap difference and update it in each loop
    y_chn = erf['CHN PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'CHN PMI'])
    y_us = erf['USA PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'USA PMI'])
    erf['y_diff'] = y_chn - y_us
    
    reg_trsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'pi_diff','y_diff']], missing='drop')
    results_sub = reg_trsub.fit()
    
    # error = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']]) - erf.loc[size_subsample+i, 'dep_1']
    
    fcst_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff']]])
    dep_1_tr[i]=erf.loc[size_subsample+i, 'dep_1']
    error = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff']]]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_error_tr[i] = error

#Use the percentage of correct directional forecast as the criterion
fcst_tr_direction=np.where(fcst_tr>=0,1,0)
dep_1_tr_direction=np.where(dep_1_tr>=0,1,0)
percentage_of_correct_directional_forecast_tr=np.mean(fcst_tr_direction==dep_1_tr_direction)
print(percentage_of_correct_directional_forecast_tr)

0.640449438202


In [32]:
dep_1_tr

array([  5.22617214e-01,  -1.61938469e+00,  -1.53824357e-01,
        -2.18448505e-01,  -8.04582163e-01,  -5.08637316e-01,
        -2.11177277e-01,  -2.86332402e-01,  -8.79335292e-01,
        -2.23360512e-01,  -1.99134064e-01,  -4.24287169e-01,
        -8.96279839e-01,  -4.99153419e-01,  -4.98494509e-01,
         3.93008437e-01,  -7.47882782e-01,   1.68088587e-01,
        -3.11027434e-01,   3.81370077e-02,  -2.48150927e-01,
         9.00578457e-01,  -1.67451304e-01,   1.12191781e-01,
         2.03519858e-01,  -6.14855883e-02,  -6.45510579e-01,
        -1.74750231e-01,  -5.88483241e-02,  -9.55033897e-02,
        -2.54829823e-02,  -1.43462900e-01,  -7.70238481e-01,
        -6.64497139e-01,  -1.45651101e-02,   1.61845033e-03,
        -1.27938348e-01,  -3.71786870e-01,  -8.95000263e-02,
        -1.62932826e-01,  -5.82205187e-01,   1.32766228e-01,
         2.68272097e-01,   5.00265841e-01,   9.58562555e-02,
         1.86574785e-01,  -2.71053460e-01,   2.38630666e-01,
        -4.54095790e-02,

In [43]:
#the random walk model
fcst_rw = erf.loc[size_subsample: size_sample-1, 'dep']
fcst_rw_direction=np.where(fcst_rw>=0,1,0)
percentage_of_correct_directional_forecast_rw=np.mean(fcst_rw_direction==dep_1_tr_direction)
print(percentage_of_correct_directional_forecast_rw)

0.61797752809


## Exercise 2

In [47]:
#Taylor Rule model with REER
erf['pi_diff'] = erf['CHN CPI inflation'] - erf['USA CPI inflation']
y_chn = erf['CHN PMI'] - np.mean(erf['CHN PMI'])
y_us = erf['USA PMI'] - np.mean(erf['USA PMI'])
erf['y_diff'] = y_chn - y_us
reg_tr_reer = sm.OLS(endog=erf['dep_1'], exog=erf[['const', 'pi_diff','y_diff','RMB_REER']], missing='drop')
results = reg_tr_reer.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  dep_1   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     9.796
Date:                Fri, 02 Feb 2018   Prob (F-statistic):           6.33e-06
Time:                        16:34:55   Log-Likelihood:                -137.20
No. Observations:                 149   AIC:                             282.4
Df Residuals:                     145   BIC:                             294.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1785      0.420     -5.193      0.0

### The sign of the coefficient of REER is positive. And it is significant. The sign is consistent with our intuition.

In [53]:
size_sample = 149
size_subsample = 60

fcst_tr = np.zeros(size_sample - size_subsample)
fcst_error_tr = np.zeros(size_sample - size_subsample)
dep_1_tr = np.zeros(size_sample - size_subsample)
for i in range (size_sample - size_subsample):
    # Create a column to store the output gap difference and update it in each loop
    y_chn = erf['CHN PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'CHN PMI'])
    y_us = erf['USA PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'USA PMI'])
    erf['y_diff'] = y_chn - y_us
    
    reg_trsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'pi_diff','y_diff','RMB_REER']], missing='drop')
    results_sub = reg_trsub.fit()
    
    # error = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']]) - erf.loc[size_subsample+i, 'dep_1']
    
    fcst_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff','RMB_REER']]])
    dep_1_tr[i]=erf.loc[size_subsample+i, 'dep_1']
    error = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff','RMB_REER']]]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_error_tr[i] = error

rmse_tr_reer = np.sqrt(np.mean(fcst_error_tr**2))
print(rmse_tr_reer) 

0.7605718598


### The model produce smaller RMSE than the original Taylor Rule model.